<a href="https://colab.research.google.com/github/mantoan-thi/Wire-harness-analysis/blob/main/Comparando_chicotes_Yazaki.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [309]:
import pandas as pd

In [444]:
class comparative_analysis():
  def __init__(self,dataset):
    self.dataset = dataset[['PROD_TERM', 'OT', 'OT_DISCRIMINADA', 'CIRC_ESQ','CIR_DIR', 'TIPO', 
                            'BITOLA', 'COR', 'COMP', 'COD_OP_01_ESQ', 'TERMINAL_ESQ','ACC_ESQ_1',
                            'COD_OP_01_DIR','TERMINAL_DIR', 'ACC_DIR_1', 'SOLDA']]

  # Função retir as ot discriminadas
  def ot_disc(self):
    # Obtenha os nomes dos índices para os quais a coluna 'OT_DISCRIMINADA' tem o valor SIM
    index_ot_discriminada = self.dataset[self.dataset['OT_DISCRIMINADA']=='SIM'].index
    if index_ot_discriminada.shape[0] > 0:
      # Excluindo esses índices de linha de dataFrame
      self.dataset.drop(index_ot_discriminada, inplace=True)
    self.dataset.drop(columns=['OT_DISCRIMINADA'],axis=1, inplace=True)
    self.dataset.fillna(0,inplace=True)
    return self.dataset

  # Quantidade de circuitos por part number
  def wire_stat(self): 
    self.lista_pn = []
    pn_anterior = 0
    for pn in self.dataset['PROD_TERM']:
      if pn_anterior != pn:
        pn_anterior = pn
        qtde = self.dataset[self.dataset['PROD_TERM']==pn].shape[0]
        self.lista_pn.append([pn,qtde])
    self.lista_pn =pd.DataFrame(self.lista_pn,columns=['Part_number','Qtde'])
    self.lista_pn.sort_values(by=['Qtde'],ascending=False,inplace=True)
    self.lista_pn.reset_index(inplace=True,drop=True)


    self.qtde_pn = self.lista_pn.shape[0]
    self.menor_pn = self.lista_pn['Part_number'][-1:].values[0]
    self.maior_pn = self.lista_pn['Part_number'][0]
    self.Describ = self.lista_pn.describe()
    return self.lista_pn,self.qtde_pn,self.menor_pn,self.maior_pn,self.Describ

  def Num_compar(self):
    # Calcular a quantidade de comparações
    self.lista_pn,_,_ ,_,_= self.wire_stat()
    contador1 =0
    contador2 = 0
    Total = 0
    while contador1 < self.lista_pn['Part_number'].shape[0]:
      contador2=contador1+1
      while contador2 < self.lista_pn['Part_number'].shape[0]:
        #print('Nr:'+str(Total)+' - '+self.lista_pn['Part_number'][contador1]+' vs '+self.lista_pn['Part_number'][contador2])
        contador2+=1
        Total+=1
      contador1+=1
      contador2 = 0
    return Total



In [516]:
dataset = pd.read_excel('/content/Consulta BOM v14.xlsm',sheet_name='CIRCUITO')

In [517]:
analise = comparative_analysis(dataset)

In [518]:
analise.Num_compar()
dataset = analise.ot_disc()
lista_pn,_,_,_,_ = analise.wire_stat()

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [521]:
list_comp.columns

Index(['PROD_TERM', 'OT', 'CIRC_ESQ', 'CIR_DIR', 'TIPO', 'BITOLA', 'COR',
       'COMP', 'COD_OP_01_ESQ', 'TERMINAL_ESQ', 'ACC_ESQ_1', 'COD_OP_01_DIR',
       'TERMINAL_DIR', 'ACC_DIR_1', 'SOLDA'],
      dtype='object')

In [564]:
def analisar(list_comp):
  contar1 = 0
  contar2 = 0
  list_comp['Delta'] = 0
  while contar1 < list_comp.shape[0]:
    contar2 = contar1+1
    while contar2 < list_comp.shape[0]:
      if list_comp['PROD_TERM'].iloc[contar1]!=list_comp['PROD_TERM'].iloc[contar2]:
        # Tipo+bit+cor
        tipoA = list_comp['TIPO'].iloc[contar1]+list_comp['BITOLA'].iloc[contar1]+list_comp['COR'].iloc[contar1]
        tipoB = list_comp['TIPO'].iloc[contar2]+list_comp['BITOLA'].iloc[contar2]+list_comp['COR'].iloc[contar2]
        TAE = str(list_comp['COD_OP_01_ESQ'].iloc[contar1])+str(list_comp['TERMINAL_ESQ'].iloc[contar1])+str(list_comp['ACC_ESQ_1'].iloc[contar1])
        TAD = str(list_comp['COD_OP_01_DIR'].iloc[contar1])+str(list_comp['TERMINAL_DIR'].iloc[contar1])+str(list_comp['ACC_DIR_1'].iloc[contar1])
        TBE = str(list_comp['COD_OP_01_ESQ'].iloc[contar2])+str(list_comp['TERMINAL_ESQ'].iloc[contar2])+str(list_comp['ACC_ESQ_1'].iloc[contar2])
        TBD = str(list_comp['COD_OP_01_DIR'].iloc[contar2])+str(list_comp['TERMINAL_DIR'].iloc[contar2])+str(list_comp['ACC_DIR_1'].iloc[contar2])
        COMPA = int(list_comp['COMP'].iloc[contar1])
        COMPB = int(list_comp['COMP'].iloc[contar2])
        dif = abs(COMPA-COMPB)
        JTA = str(list_comp['SOLDA'].iloc[contar1])[0]
        JTB = str(list_comp['SOLDA'].iloc[contar2])[0]
        if tipoA == tipoB:                            # Comparando Tipo,Bit e cor
          if COMPA == COMPB or dif < 50:              # Checando dimensional
            if TAE==TBE or TAD==TBD:                  # Terminais
              list_comp['Delta'].iloc[contar1] ='COMUM' 
              list_comp['Delta'].iloc[contar2] ='COMUM'
            if TAE==TBD or TAD==TBE:                  # Terminais
              list_comp['Delta'].iloc[contar1] ='COMUM' 
              list_comp['Delta'].iloc[contar2] ='COMUM'

      contar2 +=1
    contar1+=1
    contar2 = 0
    list_comp['Delta'].loc[list_comp['Delta']==0]='DIF'
    lista_analisada = list_comp.sort_values(by=['OT'],ascending=True,inplace=True)
  return list_comp

In [574]:
# Part number que será o 'PARA'
partA = lista_pn['Part_number'][7]
partA[0:10]

'82115F570'

In [ ]:
contador = 0
for pn in lista_pn['Part_number']:
  if pn != partA:
    list_comp = dataset[(dataset['PROD_TERM']==pn) | (dataset['PROD_TERM']==partA)]
    nova_lista = analisar(list_comp)
    nova_lista.to_csv(r'/content/Analise'+pn[0:10]+' vs '+partA[0:10]+'.csv',index=False, sep=',')

In [567]:
nova_lista[nova_lista['Delta']!='COMUM']

,PROD_TERM,OT,CIRC_ESQ,CIR_DIR,TIPO,BITOLA,COR,COMP,COD_OP_01_ESQ,TERMINAL_ESQ,ACC_ESQ_1,COD_OP_01_DIR,TERMINAL_DIR,ACC_DIR_1,SOLDA,Delta
7092,82115F5750 A20,33,#67 203,C27 079,CIVUS,0.35,B,2495,0,7116-3153-02,0,D,7114-4025,7158-3009-60,0,DIF
7095,82115F5750 A20,37,%A4 453,Z8E 079,CIVUS,0.35,BR,925,BU,7409-9915,0,D,7114-4025,7158-3009-60,0,DIF
7096,82115F5750 A20,38,E0B 098,SRD 310,CIVUS,0.35,BR,2235,D,7116-5044-02,7158-3009-60,0,7116-3153-02,0,0,DIF
